In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import chromedriver_binary
from pymongo import MongoClient 
from time import sleep

% matplotlib inline

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By

In [3]:
# making connection to mongodb

client = MongoClient()
database_schoolFinds = client['db_schoolFinds']
collection_schoolsFindsPass = database_schoolFinds['collection_schoolFindsPass']
collection_schoolsFindsFail = database_schoolFinds['collection_schoolFindsFail']

In [4]:
driver = webdriver.Chrome()
driver.get("http://www.schoolfinds.com/")
sleep(5)
driver.maximize_window()

In [5]:
# finding all the cities links

driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
sleep(3)

selectCityButton = driver.find_element_by_xpath("//div[@class='header-content visible-desktop']//div[@class='span4']//a[2]")
selectCityButton.click()
sleep(2)

allCityLinks = driver.find_elements_by_xpath("//div[@class='modal-body']//ul[@class='last']//a")

closeButton = driver.find_element_by_xpath('//*[@id="popup"]/footer/button')
closeButton.click()
sleep(2)

In [6]:
all_city_links_list = []

for eachCity in allCityLinks:
    all_city_links_list.append(eachCity.get_attribute('href'))

In [24]:
len(allCityLinks)

241

In [27]:
# 0-28 done  # excluding 28
# 28-100 # crawler - II
# 100 - 150 # crawler - III
# 150 - 200 # crawler - IV
# 200 - 242 # crawler - V

In [26]:
all_city_links_list = all_city_links_list[0:28]

['http://www.schoolfinds.com/delhi/narela.html',
 'http://www.schoolfinds.com/delhi/bhajan-pura.html',
 'http://www.schoolfinds.com/delhi/new-delhi.html',
 'http://www.schoolfinds.com/delhi/dwarka.html',
 'http://www.schoolfinds.com/delhi/vasant-vihar.html',
 'http://www.schoolfinds.com/delhi/south-west-delhi.html',
 'http://www.schoolfinds.com/delhi/north-west-delhi.html',
 'http://www.schoolfinds.com/delhi/janak-puri.html',
 'http://www.schoolfinds.com/delhi/east-new-delhi.html',
 'http://www.schoolfinds.com/delhi/nangloi.html',
 'http://www.schoolfinds.com/delhi/lajpat-nagar.html',
 'http://www.schoolfinds.com/delhi/mehrauli.html',
 'http://www.schoolfinds.com/delhi/rohini.html',
 'http://www.schoolfinds.com/delhi/pitampura.html',
 'http://www.schoolfinds.com/delhi/south-delhi.html',
 'http://www.schoolfinds.com/delhi/shahdara.html',
 'http://www.schoolfinds.com/delhi/mayur-vihar.html',
 'http://www.schoolfinds.com/delhi/uttam-nagar.html',
 'http://www.schoolfinds.com/delhi/badarpur

In [ ]:
# # optional

# for eachCity in driver.find_elements_by_xpath("//div[@class='modal-body']//ul[@class='last']//li//a"):
#     print(eachCity.get_attribute('title'))

In [ ]:
for eachCity in all_city_links_list:
    
    
    
    driver.get(eachCity)
    sleep(5)
    
    noOfSchoolsTotal = int(re.sub(r"\D", '', str(driver.find_element_by_xpath(r"//div[@class='product-count']").text)))  # 196
    print("Total no of schools are : ", noOfSchoolsTotal)
    
    noOfOlsEstimated = int(noOfSchoolsTotal/10)   # 19 suppose
    print("possible number of ols estimated : ", noOfOlsEstimated)
    
    print("scrolling to page end!!! : for ",noOfOlsEstimated, " times")
    
    for i in range(noOfOlsEstimated):   # cover ols created
        
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        sleep(3)
        
        
        
    if (noOfSchoolsTotal < 10):
        print("no of schools are less than 10")
        
        noOfLisPresent_in_this_ol = len(driver.find_elements_by_xpath(r"//ol[@id='products-list'][1]" + "//li[contains(@class,'item')]"))
        print()
        print("No of lis  presentin this single ol :", noOfLisPresent_in_this_ol)
        
        for j in range(1, noOfLisPresent_in_this_ol + 1):  # for this "ol", specifically, go by individual "li" under this "ol"
            
            try:
                schoolAboutDict = {}

                schoolName = driver.find_element_by_xpath("//ol[@id='products-list'][" + str(1) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//h2").text
                schoolAboutDict['schoolName'] = schoolName

                schoolAddress = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(1) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span").text
                schoolAddressRefined = re.findall(r"Address(.*?)\nEmail", schoolAddress)[0]
                schoolAboutDict['schoolAddress'] = schoolAddressRefined

                schoolEmail = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(1) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][1]").text
                schoolAboutDict['schoolEmail'] = schoolEmail

                schoolContact = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(1) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][2]").text
                schoolAboutDict['schoolContact'] = schoolContact

                schoolURL = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(1) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][4]//a").get_attribute('href')
                schoolAboutDict['schoolURL'] = schoolURL
                
                if (collection_schoolsFindsPass.find_one({"schoolName":schoolName, "schoolAddress":schoolAddressRefined})):
                    print("Already Present in the db")
                    continue

                collection_schoolsFindsPass.insert_one(schoolAboutDict)
                sleep(2)
                
                print("School Details : ", schoolAboutDict)
                print()
                
            except:
                
                print("Error for the head url city : ", eachCity)
                
                schoolDictFail = {}
                schoolDictFail['Error City URL'] = str(eachCity)
                
                if len(str(schoolName) > 0):
                    schoolDictFail['Error School Name'] = str(schoolName)
                
                collection_schoolsFindsFail.insert_one(schoolDictFail)
                
                
        
    else:
        
        if (noOfSchoolsTotal > noOfOlsEstimated*10):   # 196 > 190 
            print("WE NEED AN EXTRAA ITERATION!!!!  AS  :", noOfSchoolsTotal," > " ,noOfOlsEstimated*10)
        
            for i in range(1, noOfOlsEstimated + 2):    # go by "ol" number 1,2,3,4,5,6,7,8,9,.....18,19,20

                print("-------------------------STARTING FOR OL---------------------------- : ", i)
                print()

                noOfLisPresent_in_this_ol = len(driver.find_elements_by_xpath(r"//ol[@id='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')]"))
                print()
                print("No of lis  presentin this ol :", noOfLisPresent_in_this_ol)

                for j in range(1, noOfLisPresent_in_this_ol + 1):  # for this "ol", specifically, go by individual "li" under this "ol"

                    try:
                        schoolAboutDict = {}

                        schoolName = driver.find_element_by_xpath("//ol[@id='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//h2").text
                        schoolAboutDict['schoolName'] = schoolName

                        schoolAddress = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span").text
                        schoolAddressRefined = re.findall(r"Address(.*?)\nEmail", schoolAddress)[0]
                        schoolAboutDict['schoolAddress'] = schoolAddressRefined

                        schoolEmail = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][1]").text
                        schoolAboutDict['schoolEmail'] = schoolEmail

                        schoolContact = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][2]").text
                        schoolAboutDict['schoolContact'] = schoolContact

                        schoolURL = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][4]//a").get_attribute('href')
                        schoolAboutDict['schoolURL'] = schoolURL

                        if (collection_schoolsFindsPass.find_one({"schoolName":schoolName, "schoolAddress":schoolAddressRefined})):
                            print("Already Present in the db")
                            continue


                        collection_schoolsFindsPass.insert_one(schoolAboutDict)
                        print("School Inserted with success")
                        sleep(2)

                        print("School Details : ", schoolAboutDict)
                        print()



                    except:

                        print("Error for the head url city : ", eachCity)

                        schoolDictFail = {}
                        schoolDictFail['Error City URL'] = str(eachCity)

                        if len(str(schoolName) > 0):
                            schoolDictFail['Error School Name'] = str(schoolName)

                        collection_schoolsFindsFail.insert_one(schoolDictFail)

        else:
            print("schools can be exact multiples of 10 ")
            
            for i in range(1, noOfOlsEstimated + 1):    # go by "ol" number 1,2,3,4,5,6,7,8,9,.....18,19

                print("-------------------------STARTING FOR OL---------------------------- : ", i)
                print()

                noOfLisPresent_in_this_ol = len(driver.find_elements_by_xpath(r"//ol[@id='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')]"))
                print()
                print("No of lis  presentin this ol :", noOfLisPresent_in_this_ol)

                for j in range(1, noOfLisPresent_in_this_ol + 1):  # for this "ol", specifically, go by individual "li" under this "ol"

                    try:
                        schoolAboutDict = {}

                        schoolName = driver.find_element_by_xpath("//ol[@id='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//h2").text
                        schoolAboutDict['schoolName'] = schoolName

                        schoolAddress = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span").text
                        schoolAddressRefined = re.findall(r"Address(.*?)\nEmail", schoolAddress)[0]
                        schoolAboutDict['schoolAddress'] = schoolAddressRefined

                        schoolEmail = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][1]").text
                        schoolAboutDict['schoolEmail'] = schoolEmail

                        schoolContact = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][2]").text
                        schoolAboutDict['schoolContact'] = schoolContact

                        schoolURL = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][4]//a").get_attribute('href')
                        schoolAboutDict['schoolURL'] = schoolURL

                        if (collection_schoolsFindsPass.find_one({"schoolName":schoolName, "schoolAddress":schoolAddressRefined})):
                            print("Already Present in the db")
                            continue


                        collection_schoolsFindsPass.insert_one(schoolAboutDict)
                        print("School Inserted with success")
                        sleep(2)

                        print("School Details : ", schoolAboutDict)
                        print()



                    except:

                        print("Error for the head url city : ", eachCity)

                        schoolDictFail = {}
                        schoolDictFail['Error City URL'] = str(eachCity)

                        if len(str(schoolName) > 0):
                            schoolDictFail['Error School Name'] = str(schoolName)

                        collection_schoolsFindsFail.insert_one(schoolDictFail)

        

Total no of schools are :  190
possible number of ols estimated :  19
scrolling to page end!!! : for  19  times
schools can be exact multiples of 10 
-------------------------STARTING FOR OL---------------------------- :  1


No of lis  presentin this ol : 10
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
-------------------------STARTING FOR OL---------------------------- :  2


No of lis  presentin this ol : 10
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
-------------------------STARTING FOR OL---------------------------- :  3


No of lis  presentin this ol : 10


Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
-------------------------STARTING FOR OL---------------------------- :  4


No of lis  presentin this ol : 10
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
-------------------------STARTING FOR OL---------------------------- :  5


No of lis  presentin this ol : 10
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db


Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
-------------------------STARTING FOR OL---------------------------- :  26


No of lis  presentin this ol : 10
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
-------------------------STARTING FOR OL---------------------------- :  27


No of lis  presentin this ol : 10
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
Already Present in the db
-------------------------STARTING FOR OL----------

School Inserted with success
School Details :  {'schoolName': 'SARASWATI VIDYA MANDIR', 'schoolAddress': ': DENDAYALPURAM SAMBHAL MORADABAD UTTAR PRADESH -244302', 'schoolEmail': 'Email: shobhamathur27@gmail.com', 'schoolContact': 'Contact Number: 05923 -231358', 'schoolURL': 'http://www.schoolfinds.com/saraswati-vidya-mandir-12552.html', '_id': ObjectId('5b895a4eedf2ee14504cdb84')}

School Inserted with success
School Details :  {'schoolName': 'SNEH LATA VIDYA MANDIR', 'schoolAddress': ': Sambhal Road- Maulagarh- Chandausi- Distt Moradabad- Uttar Pradesh -202412', 'schoolEmail': 'Email: sachin.naturezone@gmail.com', 'schoolContact': 'Contact Number: 5921312752', 'schoolURL': 'http://www.schoolfinds.com/sneh-lata-vidya-mandir.html', '_id': ObjectId('5b895a51edf2ee14504cdb85')}

School Inserted with success
School Details :  {'schoolName': "ST MARY'S SCHOOL", 'schoolAddress': ': SHANTIPURAM MANDI ROAD PO SAMBHAL DISTT MORADABAD UTTAR PRADESH -244302', 'schoolEmail': 'Email: dhtsvm@airte

Already Present in the db
Already Present in the db
School Inserted with success
School Details :  {'schoolName': 'BSS PUBLIC SCHOOL', 'schoolAddress': ': AMOL NAGAR- RAIBAREILLI- UTTAR PRADESH -229001', 'schoolEmail': 'Email: bsseducation@rediffmail.com', 'schoolContact': 'Contact Number: 95120 -0535-2203242', 'schoolURL': 'http://www.schoolfinds.com/bss-public-school.html', '_id': ObjectId('5b895ad5edf2ee14504cdb92')}

School Inserted with success
School Details :  {'schoolName': 'CHINMAYA VIDYALAYA', 'schoolAddress': ': UNCHAHAR RAIBEREILLY UTTAR PRADESH -229406', 'schoolEmail': 'Email: amitavaghosh2k1@rediffmail.com', 'schoolContact': 'Contact Number: 05311 -277207', 'schoolURL': 'http://www.schoolfinds.com/chinmaya-vidyalaya-18710.html', '_id': ObjectId('5b895ad9edf2ee14504cdb93')}

School Inserted with success
School Details :  {'schoolName': 'G.K. CITY MONTESSORI SCHOOL', 'schoolAddress': ': Karamchari Nagar Izatnagar Bareilly-243 122', 'schoolEmail': 'Email: gkcms.bly@gmail.com

...

School Details :  {'schoolName': 'UNION PUBLIC SCHOOL', 'schoolAddress': ': Vijay Garh Road Sasni- Distt Hathras- Uttar Pradesh -204216', 'schoolEmail': 'Email: unionpublicschool@yahoo.com', 'schoolContact': 'Contact Number: 05722-220513', 'schoolURL': 'http://www.schoolfinds.com/union-public-school.html', '_id': ObjectId('5b896887edf2ee14504cdef7')}

Total no of schools are :  9
possible number of ols estimated :  0
scrolling to page end!!! : for  0  times
no of schools are less than 10

No of lis  presentin this single ol : 9
School Details :  {'schoolName': 'BLOOMING BUDS PUBLIC SCHOOL', 'schoolAddress': ': MAU SHAMSHABAD PO DEVTAHA BALRAMPUR GONDA UTTAR PRADESH -271203', 'schoolEmail': 'Email: jagadish_panda@rediffmail.com', 'schoolContact': 'Contact Number: 05262 -250049', 'schoolURL': 'http://www.schoolfinds.com/blooming-buds-public-school.html', '_id': ObjectId('5b896895edf2ee14504cdef8')}

School Details :  {'schoolName': 'FATIMA SCHOOL', 'schoolAddress': ': circular road barga

School Inserted with success
School Details :  {'schoolName': 'THE ADITYA BIRLA PUBLIC SCHOOL', 'schoolAddress': ': P O RENUKOOT SONEBHADRA DISTT UTTAR PRADESH -231217', 'schoolEmail': 'Email: ois_palwal@yahoo.in', 'schoolContact': 'Contact Number: 179', 'schoolURL': 'http://www.schoolfinds.com/the-aditya-birla-public-school-11047.html', '_id': ObjectId('5b8968f0edf2ee14504cdf0c')}

School Inserted with success
School Details :  {'schoolName': "ST. JOSEPH'S SCHOOL", 'schoolAddress': ': SHAKTINAGAR P O SONEBHADRA DISTT UTTAR PRADESH -231222', 'schoolEmail': 'Email: ursulinegreaternoida@yahoo.com', 'schoolContact': 'Contact Number: 05444 -32918', 'schoolURL': 'http://www.schoolfinds.com/st-joseph-s-school-18911.html', '_id': ObjectId('5b8968f2edf2ee14504cdf0d')}

School Inserted with success
School Details :  {'schoolName': 'ST. JOSEPH CONVENT SCHOOL', 'schoolAddress': ': ROBERTS GANJ DISTT SONEBHADRA UTTAR PRADESH -231216', 'schoolEmail': 'Email: mail@carmelagiri.com', 'schoolContact': 

School Inserted with success
School Details :  {'schoolName': 'ST R C SCIENTIFIC CONVENT SCHOOL', 'schoolAddress': ': ST R C COLONY KAIRANA DISTT MUZAFFARNAGAR UTTAR PRADESH -247774', 'schoolEmail': 'Email: goodhope_school@yahoo.com', 'schoolContact': 'Contact Number: 01398 -267297', 'schoolURL': 'http://www.schoolfinds.com/st-r-c-scientific-convent-school.html', '_id': ObjectId('5b896943edf2ee14504cdf21')}

School Inserted with success
School Details :  {'schoolName': 'ST. DRONA PUBLIC SCHOOL', 'schoolAddress': ': Kairana Road- Kandhla Dehat- Distt Muzaffarnagar- Uttar Pradesh -247775', 'schoolEmail': 'Email: stdps2010@gmail.com', 'schoolContact': 'Contact Number: 0591 -01292-222245- 222246', 'schoolURL': 'http://www.schoolfinds.com/st-drona-public-school.html', '_id': ObjectId('5b896946edf2ee14504cdf22')}

School Inserted with success
School Details :  {'schoolName': 'ST. R C SCIENTIFIC CONVENT SCHOOL', 'schoolAddress': ': NEW MANDI STHAL MAJRA ROAD SHAMLI MUZAFFARNAGAR UTTAR PRADESH

School Inserted with success
School Details :  {'schoolName': "ST. ANTHONY'S SCHOOL FOR GIRLS", 'schoolAddress': ': ANTONIAN CAMPUS RAHE MURTAZA CIVIL LINES RAMPUR UTTAR PRADESH -244901', 'schoolEmail': 'Email: hinduindiranagar@dataone.in', 'schoolContact': 'Contact Number: 044 -2350222', 'schoolURL': 'http://www.schoolfinds.com/st-anthony-s-school-for-girls.html', '_id': ObjectId('5b89698eedf2ee14504cdf35')}

School Inserted with success
School Details :  {'schoolName': "ST XAVIER'S HIGH SCHOOL", 'schoolAddress': ': BISHNU PURA DISTT BALRAMPUR UTTAR PRADESH -271201', 'schoolEmail': 'Email:', 'schoolContact': 'Contact Number: 05263 -234754', 'schoolURL': 'http://www.schoolfinds.com/st-xavier-s-high-school-18167.html', '_id': ObjectId('5b896991edf2ee14504cdf36')}

School Inserted with success
School Details :  {'schoolName': 'R A N PUBLIC SCHOOL', 'schoolAddress': ': P.O.KAUNSAL GANJ BILASPUR RAMPUR UTTAR PRADESH -244921', 'schoolEmail': 'Email: anwar@lw1.vsnl.net.in', 'schoolContact': 

School Inserted with success
School Details :  {'schoolName': 'M L GOVT BOYS SR SEC SCHOOL', 'schoolAddress': ': narela delhi -110040', 'schoolEmail': 'Email:', 'schoolContact': 'Contact Number: 25314900 -27452368', 'schoolURL': 'http://www.schoolfinds.com/m-l-govt-boys-sr-sec-school.html', '_id': ObjectId('5b8969deedf2ee14504cdf4a')}

School Inserted with success
School Details :  {'schoolName': 'KENDRIYA VIDYALAYA', 'schoolAddress': ': sec a-5- pocket - 11- narela- delhi -110040', 'schoolEmail': 'Email:', 'schoolContact': 'Contact Number: 011 -27787279', 'schoolURL': 'http://www.schoolfinds.com/kendriya-vidyalaya-108636.html', '_id': ObjectId('5b8969e2edf2ee14504cdf4b')}

School Inserted with success
School Details :  {'schoolName': 'KASTURI RAM INTERNATIONAL SCHOOL', 'schoolAddress': ': SECTOR A - 10- NARELA- DISTT NORTH DELHI- DELHI -110040', 'schoolEmail': 'Email: satyendrakj@rediffmail.com', 'schoolContact': 'Contact Number: 011 -27781911', 'schoolURL': 'http://www.schoolfinds.co

In [ ]:
# for eachCity in all_city_links_list:
    
    
    
#     driver.get(eachCity)
#     sleep(5)
    
#     noOfSchoolsTotal = int(re.sub(r"\D", '', str(driver.find_element_by_xpath(r"//div[@class='product-count']").text)))  # 196
#     print("Total no of schools are : ", noOfSchoolsTotal)
    
#     noOfOlsEstimated = int(noOfSchoolsTotal/10)   # 19 suppose
#     print("possible number of ols estimated : ", noOfOlsEstimated)
    
#     print("scrolling to page end!!! : for ",noOfOlsEstimated, " times")
#     for i in range(noOfOlsEstimated):   # cover ols created
        
#         driver.find_element_by_tag_name('body').send_keys(Keys.END)
#         sleep(3)
        
#     if (noOfSchoolsTotal < 10):
        
#         noOfLisPresent_in_this_ol = len(driver.find_elements_by_xpath(r"//ol[@id='products-list'][1]" + "//li[contains(@class,'item')]"))
#         print()
#         print("No of lis  presentin this single ol :", noOfLisPresent_in_this_ol)
        
#         for j in range(1, noOfLisPresent_in_this_ol + 1):  # for this "ol", specifically, go by individual "li" under this "ol"
            
#             try:
#                 schoolAboutDict = {}

#                 schoolName = driver.find_element_by_xpath("//ol[@id='products-list'][" + str(1) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//h2").text
#                 schoolAboutDict['schoolName'] = schoolName

#                 schoolAddress = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(1) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span").text
#                 schoolAddressRefined = re.findall(r"Address(.*?)\nEmail", schoolAddress)[0]
#                 schoolAboutDict['schoolAddress'] = schoolAddressRefined

#                 schoolEmail = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(1) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][1]").text
#                 schoolAboutDict['schoolEmail'] = schoolEmail

#                 schoolContact = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(1) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][2]").text
#                 schoolAboutDict['schoolContact'] = schoolContact

#                 schoolURL = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(1) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][4]//a").get_attribute('href')
#                 schoolAboutDict['schoolURL'] = schoolURL
                
#                 if (collection_schoolsFindsPass.find_one({"schoolName":schoolName, "schoolAddress":schoolAddressRefined})):
#                     print("Already Present in the db")
#                     continue

#                 collection_schoolsFindsPass.insert_one(schoolAboutDict)
#                 sleep(2)
                
#                 print("School Details : ", schoolAboutDict)
#                 print()
                
#             except:
                
#                 print("Error for the head url city : ", eachCity)
                
#                 schoolDictFail = {}
#                 schoolDictFail['Error City URL'] = str(eachCity)
                
#                 if len(str(schoolName) > 0):
#                     schoolDictFail['Error School Name'] = str(schoolName)
                
#                 collection_schoolsFindsFail.insert_one(schoolDictFail)
                
                
        
#     else:
    
#         for i in range(1, noOfOlsEstimated + 1):    # go by "ol" number 1,2,3,4,5,6,7,8,9,.....18,19

#             print("-------------------------STARTING FOR OL---------------------------- : ", i)
#             print()

#             noOfLisPresent_in_this_ol = len(driver.find_elements_by_xpath(r"//ol[@id='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')]"))
#             print()
#             print("No of lis  presentin this ol :", noOfLisPresent_in_this_ol)

#             for j in range(1, noOfLisPresent_in_this_ol + 1):  # for this "ol", specifically, go by individual "li" under this "ol"

#                 try:
#                     schoolAboutDict = {}

#                     schoolName = driver.find_element_by_xpath("//ol[@id='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//h2").text
#                     schoolAboutDict['schoolName'] = schoolName

#                     schoolAddress = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span").text
#                     schoolAddressRefined = re.findall(r"Address(.*?)\nEmail", schoolAddress)[0]
#                     schoolAboutDict['schoolAddress'] = schoolAddressRefined

#                     schoolEmail = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][1]").text
#                     schoolAboutDict['schoolEmail'] = schoolEmail

#                     schoolContact = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" +  "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][2]").text
#                     schoolAboutDict['schoolContact'] = schoolContact

#                     schoolURL = driver.find_element_by_xpath("//ol[@class='products-list'][" + str(i) + "]" + "//li[contains(@class,'item')][" + str(j) + "]//div[@class='desc std']//span//div[@class='std' and @itemprop='description'][4]//a").get_attribute('href')
#                     schoolAboutDict['schoolURL'] = schoolURL

#                     if (collection_schoolsFindsPass.find_one({"schoolName":schoolName, "schoolAddress":schoolAddressRefined})):
#                         print("Already Present in the db")
#                         continue
                        
                        
#                     collection_schoolsFindsPass.insert_one(schoolAboutDict)
#                     print("School Inserted with success")
#                     sleep(2)

#                     print("School Details : ", schoolAboutDict)
#                     print()



#                 except:

#                     print("Error for the head url city : ", eachCity)

#                     schoolDictFail = {}
#                     schoolDictFail['Error City URL'] = str(eachCity)
                    
#                     if len(str(schoolName) > 0):
#                         schoolDictFail['Error School Name'] = str(schoolName)

#                     collection_schoolsFindsFail.insert_one(schoolDictFail)


        